### Unit 08: Final Energy Demands
## Welcome to your 8-Region-Renewable-Energy System Model of Niger

#### Exercise 2 - Adding and changing hydrogen demands:

Based on the ESM of Execise 1.1 let us model an energy system where Niger's regions have seen the first industries using green hydrogen for fertilizer production, and the first pilot projects using hydrogen in the transportation sector are underway.

a) Add the perepared hydrogen demand excel: "hydrogen_dem2030_NER.xlsx" as an input to the model 

b) Finally adding the hydrogen demand as a "sink" something went wrong. Can you help to add the commodity that is missing?
Hint: You can find all commodities earlier in the code!

c) Run the model! 
What happens to the sources after adding hydrogen in comparison of before adding the hydrogen demand? Check the optimization summary.

#### 1. Import FINE and further modules to run the model 

In [ ]:
import FINE as fn
import geopandas as gpd
import pandas as pd
import numpy as np
import geokit as gk
from os.path import dirname, abspath, join
import matplotlib.pyplot as plt
import os

%matplotlib inline
%load_ext autoreload
%autoreload 2



#### 2. Set paths as input data for the model 

In [ ]:
#set paths
cwd = os.getcwd()
data_dir = join(cwd, "data")

path_to_regions = join(data_dir, "regions", "region_shape_NER.shp")

path_to_pv_ts = join(data_dir, "sources", "solar_ts_NER.csv")
path_to_onshore_ts = join(data_dir, "sources", "onshore_ts_NER.csv")

path_to_pv_cap = join(data_dir, "sources", "solar_cap_NER.csv")
path_to_onshore_cap = join(data_dir, "sources", "onshore_cap_NER.csv")

path_to_el_dem = join(data_dir, "sinks", "electricity_dem_NER.xlsx")
path_to_h2_dem = join(data_dir, "sinks", "?.xlsx")

#### Hint: Where is the "data" folder? 

In [ ]:
## Show path to data folder
data_dir

#### 2.1 Set up the regions and the commodities of the model

In [ ]:
locations_shape = gpd.read_file(path_to_regions)
locations = locations_shape.GID_1.to_list() #will be ["NER.1_1", "NER.2_1", ... "NER.8_1"]

commodities = {"electricity", "hydrogen_gas"}
commodityUnitsDict = {
                "electricity": r"GW$_{el}$",
                "hydrogen_gas": r"GW$_{H_{2},LHV}$",
            }
            

#### 3 Set up the energy system model class

In [ ]:
#Set up esm Model

esM = fn.EnergySystemModel(
    locations=set(locations),
    commodities=commodities,
    numberOfTimeSteps=8760, #hours per year
    commodityUnitsDict=commodityUnitsDict,
    hoursPerTimeStep=1, #time step is one hour
    costUnit="1e9 Euro",
    lengthUnit="km",
    verboseLogLevel=0, #what is printed, just keep it
)


#### 4 Add the electricity "sources" to the model

Wind Onshore

In [ ]:
# to add pv, the maximum capacity and the time series of each region must first be loaded as input data
pv_time_series=pd.read_csv(path_to_pv_ts, index_col=[0]).reset_index(drop=True) #capacity factor [1]
pv_capacity_max=pd.read_csv(path_to_pv_cap, index_col=[0])['capacity_kW'] / 1e6 #capacity [GW]
# add PV
esM.add(
    fn.Source(
        esM=esM, 
        name="PV", 
        commodity="electricity", 
        hasCapacityVariable=True,
        operationRateMax=pv_time_series,
        capacityMax=pv_capacity_max,
        investPerCapacity=0.45, #1e9EUR/GW, 2030
        opexPerCapacity=0.017*0.45, #1e9EUR/a
        interestRate=0.08,  #1
        economicLifetime=20, #a
        ),
)


Solar PV

In [ ]:
# to add wind, the maximum capacity and the time series of each region must first be loaded as input data
onshore_time_series=pd.read_csv(path_to_onshore_ts, index_col=[0]).reset_index(drop=True) #capacity factor [1]
onshore_capacity_max=pd.read_csv(path_to_onshore_cap, index_col=[0])['capacity_kW'] / 1e6 #capacity [GW]
#add Wind Onshore
esM.add(
    fn.Source(
        esM=esM, 
        name="Onshore", 
        commodity="electricity", 
        hasCapacityVariable=True,
        operationRateMax=onshore_time_series,
        capacityMax=onshore_capacity_max,
        investPerCapacity=1.13, #1e9EUR/GW, 2030
        opexPerCapacity=0.025*1.13, #1e9EUR/a
        interestRate=0.08, #1
        economicLifetime=20, #years
        ),
)

4.1 Add super expensive power source, so that the electricity demand also can be compensated when the sun is not shining and the wind is not blowing. If you would not add this, the model would not be solveable! 

In [ ]:
# Add SuperExpensiveElectricity
esM.add(
    fn.Source(
        esM=esM, 
        name="SuperExpensiveElectricity", 
        commodity="electricity", 
        hasCapacityVariable=True,
        investPerCapacity=10e10, 
        opexPerCapacity=10e10,
        opexPerOperation=10e10, 
        interestRate=0.08, 
        economicLifetime=20, #years
        ),
)

#### 5 Add the "Conversion" class electrolyzer

In [ ]:
#add Electrolyzer
esM.add(
fn.Conversion(
    esM=esM,
    name="electrolyzer",
    physicalUnit=r"GW$_{el}$",
    commodityConversionFactors={"electricity": -1, "hydrogen_gas": 0.7},
    hasCapacityVariable=True,
    investPerCapacity=0.5,
    opexPerCapacity=0.025,
    interestRate=0.08,
    economicLifetime=10,
    )
)

#### 6 Add the Demand

Electricity Demand

In [ ]:
#add electricity Demands
electricity_demand_operationRateFix=pd.read_excel(path_to_el_dem, index_col=[0], engine="openpyxl")*3 #elec demand GW

esM.add(
    fn.Sink(
        esM=esM, 
        name="electricity_demand", 
        commodity="electricity",
        hasCapacityVariable=False, 
        operationRateFix=electricity_demand_operationRateFix, #GW
    ),
)

# Hint
print(f"Here you can find the electricity demands: {path_to_el_dem}") 

Hydrogen Demand

In [ ]:
#add hydrogen Demands
hydrogen_demand_operationRateFix=pd.read_excel(path_to_h2_dem, index_col=[0], engine="openpyxl")*3 #elec demand GW

esM.add(
    fn.Sink(
        esM=esM, 
        name="hydrogen_demand", 
        commodity="?",
        hasCapacityVariable=False, 
        operationRateFix=hydrogen_demand_operationRateFix, #GW
    )
)

# Hint
print(f"Here you can find the hydrogen demands: {path_to_h2_dem}") 

#### 7 Right now you have 8760 time steps. Aggregation to 7 typical time periods with the "aggregateTemporally" method

In [ ]:
# use time aggregation tool to reduce the calculation time to 7 typical time periods
esM.aggregateTemporally(numberOfTypicalPeriods=7, segmentation=True)

#### 8 Start the optimization

In [ ]:
#Optimize the model:
print('Optimize')
esM.optimize(
    timeSeriesAggregation=True,
    optimizationSpecs="",
    solver="glpk"
)
print('Optimization done!')

### Results:

In [ ]:
esM.getOptimizationSummary("SourceSinkModel", outputLevel=2)

In [ ]:
esM.getOptimizationSummary("ConversionModel", 2)

## Plots

In [ ]:
## Show GID_1 Regions in Niger
fn.plotLocations(path_to_regions, plotLocNames=True, indexColumn="GID_1")

#### 1 Installed electrolyzer capacity

In [ ]:
fn.plotLocationalColorMap(
    esM, "electrolyzer", path_to_regions, "GID_1", perArea=False
)